# AI Fundamental Project: Inception V3

 #### **프로젝트 목적: Inception V3를 활용한 음식 카데고리 분류**

참고사항
*   Python 3.10 Library
*   Tensorflow 2.15.0
*   T4 GPU Hardware
*   Dataset: iFood - 2019 at FGVC6



## Inception V3 배경

논문: Rethinking the Inception Architecture for Computer Vision [논문 링크](https://arxiv.org/abs/1512.00567)

#### Inception V3의 특장점:

*   더 적은 파라미터 수로 깊은 층을 만들 수 있는 구조. (42-layer 신경망)
*   앙상블 기법을 활용한 top-1 17.2% error, top-5 3.58% error를 달성 (ILSVR 2012 dataset 기준) -> VGGNet와 비슷한 연상량



# 1. 파일 및 데이터 불러오기

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#라이브러리 불러오기
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'keras'

In [7]:
#이미 학습된 모델: Pre-trained model 불러오기
model = InceptionV3(weights = 'imagenet', include_top = True)

96112376/96112376 [==============================] - 1s 0us/step


In [8]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalizati

In [15]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [16]:
!ls

'Colab Notebooks'   Datasets	  ifood-2019-fgvc6.zip	 train_set.zip	 vgg_image.png
 dataset	    FoodDataset   test_set.zip		 val_set.zip


In [17]:
!pwd

/content/drive/MyDrive


In [1]:
#모든 파일들이 정상적으로 잘 다운로드 됐는지 확인하는 방법:
from glob import glob

data_set = './FoodDataset/'
trainData = len(glob(data_set + 'train_set/*'))
# valData = len(glob(data_set + 'val_set/*'))
# testData = len(glob(data_set + 'test_set/*'))

print(trainData)
# print(valData)
# print(testData)

0
10997
106


## 데이터 분류 및 정리

train,val,test 디렉터리를 만들고 11만개의 데이터를 정리한다.

이후 train 데이터 전처리를 위해 음식 이름별로 또 한번 분류해서 디렉터리를 만들어주면 나중에 편리할 것 같아 수행해 준다.

In [9]:
import pandas as pd
import os
import shutil

#데이터 불러오기
df_class = pd.read_csv("/content/drive/MyDrive/FoodDataset/class_list.txt", sep = ' ', header = None)
df_labels = pd.read_csv("/content/drive/MyDrive/FoodDataset/train_labels.csv")
df_labels.head()

,img_name,label
0,train_101733.jpg,211
1,train_101734.jpg,211
2,train_101735.jpg,211
3,train_101736.jpg,211
4,train_101737.jpg,211


In [3]:
#class.txt 파일에 모든 음식의 카테고리가 나와 있기 때문에, 이것을 폴더로 만들어 주는 함수를 만들어 실행한다.
def createFolder(dir):
  try:
    if not os.path.exists(dir):
      os.makedirs(dir)
  except OSError:
    print ('Error: Creating directory. ' +  dir)

In [4]:
#함수 실행:
# for i in range(len(df_class)):
#   names = df_class[1][i]
#   createFolder(f'/content/drive/MyDrive/FoodDataset/food/train/{names}')

In [7]:
df_class.head()

,0,1
0,0,macaron
1,1,beignet
2,2,cruller
3,3,cockle_food
4,4,samosa


In [18]:
#해당되는 경로에 맞는 데이터를 복사하는 작업
#labels을 숫자와 이름으로 변경할 수 있는 함수.

def label_names(idx):
  return df_class[1][idx]

#데이터를 자동 이전하는 프로그램:
# for i in range(len(df_labels)):
i = 118474
f_names = df_labels['img_name'][i]
i_labels = df_labels['label'][i]
print(f_names,i_labels)
dir_names = label_names(i_labels)
print(dir_names)


#원본파일 -> dir_names / f_names 로 옮겨
org_name = f'/content/drive/MyDrive/FoodDataset/train_set/{f_names}'
cp_name = f'/content/drive/MyDrive/FoodDataset/food/train/{dir_names}/{f_names}'

shutil.copy(org_name, cp_name)
print("org_name: ", org_name)
print("cp_name: ", cp_name)

# if i % 100 == 0:
#   print(f"[{i}] {org_name} -> {cp_name}")

train_101732.jpg 123
fried_rice


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/FoodDataset/train_set/train_101732.jpg'